In [2]:
import altair as alt
import pandas as pd

recipient_df = pd.read_excel('Recipient Program Participation-2023-01-25-10-52-54.xlsx', header=9)
recipient_df.columns = recipient_df.iloc[0]
recipient_df = recipient_df.drop(recipient_df.columns[[0,2,-1]], axis=1)
recipient_df = recipient_df.iloc[1:-5]
recipient_df = recipient_df.dropna()
recipient_df

/Users/kingnutmegs/opt/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Program Participation
1,Medford,MA.,02155,Friendly Visiting
2,Boston,MA,02116,Friendly Visiting
3,Brookline,MA,02445-7022,Friendly Visiting
4,Roxbury,MA,02119-1805,Medical Escort
7,Boston,MA,02114,Friendly Visiting
...,...,...,...,...
956,Cambridge,MA,02140,Friendly Visiting
957,Boston,MA,02130,Medical Escort
958,"Dorchester,",MA,02121,Medical Escort
959,Boston,MA,02128,Friendly Visiting


In [3]:
zips = pd.read_excel('Zip.xlsx')
zips['Zip'] = zips['Zip'].apply(lambda x:'{0:0>5}'.format(x))

zips.head()



,Zip,Town
0,02101,Downtown Boston
1,02108,Beacon Hill
2,02109,Markets / Inner Harbor
3,02110,Financial District / Wharves
4,02111,Chinatown / Tufts-New England Medical Center


In [4]:
recipient_df = recipient_df.drop('Mailing City', axis = 1)
recipient_df = recipient_df.rename({'Mailing Zip/Postal Code': 'Zip'}, axis=1)
recipient_df

,Mailing State/Province,Zip,Program Participation
1,MA.,02155,Friendly Visiting
2,MA,02116,Friendly Visiting
3,MA,02445-7022,Friendly Visiting
4,MA,02119-1805,Medical Escort
7,MA,02114,Friendly Visiting
...,...,...,...
956,MA,02140,Friendly Visiting
957,MA,02130,Medical Escort
958,MA,02121,Medical Escort
959,MA,02128,Friendly Visiting


In [5]:
merged_df = pd.merge(recipient_df, zips, on='Zip')
merged_df = merged_df.rename({'Town': 'Name'}, axis=1)
merged_df

,Mailing State/Province,Zip,Program Participation,Name
0,MA,02116,Friendly Visiting,Back Bay
1,MA,02116,Medical Escort,Back Bay
2,MA,02116,Friendly Visiting,Back Bay
3,MA,02116,Medical Escort,Back Bay
4,MA,02116,Friendly Visiting,Back Bay
...,...,...,...,...
671,MA,02120,Medical Escort,Roxbury Crossing
672,MA,02120,Friendly Helping,Roxbury Crossing
673,MA,02210,Friendly Visiting,South Boston / Fort Point
674,MA,02210,Medical Escort,South Boston / Fort Point


In [7]:
click = alt.selection_multi(encodings=['color'])


# remote geojson data object
url_geojson = 'https://bostonopendata-boston.opendata.arcgis.com/datasets/boston::boston-neighborhoods.geojson?outSR=%7B%22latestWkid%22%3A2249%2C%22wkid%22%3A102686%7D'
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json'))

rec = pd.DataFrame(merged_df.groupby(['Program Participation']).count().reset_index())

rec['Program Percentage'] = round(rec['Name'] / rec['Name'].sum(), 2)

bos_map = alt.Chart(data_geojson_remote).mark_geoshape(stroke="white"
).encode(
   color=alt.condition(click, "properties.Name:N", alt.value('lightgray')),
    tooltip="properties.Name:N"
).project(
    type='identity', reflectY=True
).add_selection(
    click
)

# new_df = merged_df[merged_df['Name'] == click["properties.Name:N"]


pie = alt.Chart(rec).mark_arc().encode(
    theta='Name',
    color=alt.condition(click, "Program Participation", alt.value("lightgray")),
    tooltip=(["Program Participation",'Program Percentage'])
).transform_filter(
    click
)

chart = bos_map | pie

chart

alt.HConcatChart(...)